### 基本使用

In [8]:
import warnings
import os
from langchain.chat_models import ChatOpenAI
from  langchain_core.prompts import ChatPromptTemplate
from  langchain_core.output_parsers  import JsonOutputParser
from dotenv  import load_dotenv
warnings.filterwarnings("ignore")
load_dotenv()
api_key = os.getenv('openai_api_key')

In [9]:
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=api_key, 
    openai_api_base='https://api.deepseek.com',
    max_tokens=1024
)

In [10]:
llm.invoke("langchain是什么")

APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}

In [3]:
prompt =ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的助手"), 
    ("human", "{input}"), 
    ])

In [21]:
chain = prompt | llm
chain.invoke("langchain是什么").content

'LangChain 是一个用于构建基于大型语言模型（LLMs）的应用程序的框架。它旨在帮助开发者更轻松地集成和使用大型语言模型（如 OpenAI 的 GPT 系列）来构建复杂的应用程序。LangChain 提供了一系列工具和模块，使得开发者能够更高效地处理自然语言处理任务，如文本生成、问答系统、对话系统等。\n\n### LangChain 的主要功能包括：\n1. **链式调用（Chains）**：允许开发者将多个 LLM 调用或其他操作串联起来，形成一个复杂的处理流程。\n2. **记忆（Memory）**：支持在对话或任务中保持上下文信息，使得模型能够记住之前的交互。\n3. **代理（Agents）**：允许 LLM 根据输入动态选择和执行不同的工具或操作。\n4. **数据加载（Data Loaders）**：提供从各种数据源（如文件、数据库、API 等）加载数据的功能。\n5. **提示模板（Prompt Templates）**：帮助开发者更轻松地构建和管理提示（prompts），以便更好地引导模型生成所需的输出。\n6. **工具集成（Tools）**：支持将外部工具（如搜索引擎、数据库查询等）与 LLM 集成，扩展模型的能力。\n\n### 使用场景：\n- **问答系统**：构建能够回答用户问题的智能系统。\n- **对话系统**：开发能够进行多轮对话的聊天机器人。\n- **文本生成**：生成文章、故事、代码等。\n- **数据分析和处理**：通过自然语言与数据进行交互，执行查询或分析任务。\n\n### 示例代码：\n以下是一个简单的 LangChain 示例，展示如何使用 OpenAI 的 GPT 模型生成文本：\n\n```python\nfrom langchain import OpenAI, PromptTemplate, LLMChain\n\n# 初始化 OpenAI 模型\nllm = OpenAI(api_key="your_openai_api_key")\n\n# 定义提示模板\ntemplate = "请生成一段关于{topic}的描述。"\nprompt = PromptTemplate(template=template, input_variables=["topic"])\n\n# 创建链\nchain = 

In [4]:
outputparser =JsonOutputParser()

In [5]:
chain = prompt | llm | outputparser

In [7]:
text = 'Now that you mention it, I have to see how it is implemented in their code.'
prompt = f"将以下英文句子翻译为德语: {text}"
chain.invoke(prompt+"问题用qusetion字段返回 答案用answer字段返回")

APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from  langchain.text_splitter  import RecursiveCharacterTextSplitter
from  langchain_community.vectorstores  import Chroma
from langchain.chains  import RetrievalQA
from langchain_community.document_loaders  import PyPDFLoader

In [ ]:
loder =PyPDFLoader("./南瓜书.pdf")
pages =loder.load_and_split()
text_spliter =RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
    )
texts =text_spliter.create_documents([pages[2].page_content,pages[3].page_content])
embed=HuggingFaceEmbeddings()
db =Chroma.from_documents(texts,embed)
qa_chain =RetrievalQA.from_chain_type(llm,retriever=db.as_retriever())

qa_chain.run("南瓜书的作者是谁，以markdown格式返回")

'很抱歉，根据提供的上下文信息，我无法确定南瓜书的作者是谁。这些文本片段似乎是一些编号和页码的列表，但没有包含关于南瓜书作者的具体信息。如果您有其他关于南瓜书的背景信息或上下文，请提供更多细节，我会尽力帮助您。'

In [31]:
from langchain_core.output_parsers import StrOutputParser
prompt_topic =ChatPromptTemplate.from_template(
    "你是一个有用的助手，请你帮我生成一个关于{topic}的joke" 
)
output_parser =StrOutputParser()
chain_topic =prompt_topic | llm | output_parser
chain_topic.invoke('ice cream')

'当然可以！这里有一个关于冰淇淋的笑话：\n\n为什么冰淇淋总是很冷静？\n\n因为它总是保持“冰”度！🍦😄'

In [39]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()

prompt_topic =ChatPromptTemplate.from_template(
    "请你给我讲一个笑话，主题是{topic},字数为100字左右" 
)

chain = prompt_topic | llm | output_parser

chain.invoke('topic : 熊')

'## 《熊出没》\n\n"老张，听说你昨天去山里采蘑菇，遇到熊了？"\n\n"可不是嘛！"老张一拍大腿，"那熊可凶了，站起来比我还高！"\n\n"那你咋逃出来的？"\n\n"我啊，"老张神秘一笑，"我掏出手机，给它看了张自拍。"\n\n"啊？"\n\n"那熊一看，吓得转身就跑，边跑边喊：\'妈呀！这年头连熊都长这么丑了！\'"\n\n"......"'

In [46]:
an_prompt = ChatPromptTemplate.from_template(
    "评价一下这个笑话 {joke}输出格式如下：笑话：{joke}评价： "
    
)

t_chain = {"joke": chain} | an_prompt | llm | output_parser

t_chain.invoke('topic : 熊')

'笑话：## 《熊出没》\n\n"老张，听说你昨天去山里采蘑菇了？"老王问。\n\n"别提了，"老张摆摆手，"我正采着蘑菇呢，突然听见身后有动静。回头一看，好家伙，一只大黑熊！"\n\n"然后呢？"\n\n"我撒腿就跑啊！那熊在后面追，我跑得比兔子还快。结果你猜怎么着？"\n\n"怎么着？"\n\n"那熊突然停下来，冲我喊：\'喂，你跑啥？我是来给你送蘑菇的！\'"\n\n"啊？"\n\n"原来是我昨天落下的蘑菇篮子，被这熊捡到了。它一路追我，是想把篮子还给我呢！"\n\n老王哈哈大笑："这熊还挺讲礼貌的嘛！"\n\n评价：\n这个笑话通过一个意外的反转制造了幽默效果。故事开始时，老张遇到熊的紧张场景让人以为会是一个惊险的故事，但结局却出乎意料地温馨和搞笑。熊不仅没有威胁老张，反而表现出一种“讲礼貌”的行为，这种反差让人感到意外和有趣。笑话的节奏把握得当，对话简洁明了，最后的反转点也恰到好处，整体上是一个轻松愉快的小故事，适合在朋友间分享。'

In [47]:
compose_chain = chain | (lambda input: {'joke':input}) | an_prompt | llm | output_parser
compose_chain.invoke('大学生')

'笑话：## 《期末复习》\n\n"完了完了，明天就考试了！"室友小王抱着脑袋哀嚎。\n\n我淡定地翻开崭新的课本："别慌，我有个绝招。"\n\n"什么绝招？"\n\n"把书放在枕头底下，知识会通过渗透作用进入大脑。"\n\n"真的假的？"\n\n"当然是真的，我高中就这么干的。"\n\n"那你高考考得怎么样？"\n\n"......所以我现在跟你一个学校。"\n\n评价：\n1. **幽默感**：这个笑话通过夸张的手法，将“把书放在枕头底下”这种不切实际的复习方法描述为“绝招”，制造了一种荒诞的幽默感。最后的反转“所以我现在跟你一个学校”更是点睛之笔，揭示了这种方法的无效性，令人忍俊不禁。\n\n2. **共鸣感**：这个笑话抓住了学生群体在考试前的焦虑和临时抱佛脚的心理，容易引起学生群体的共鸣。尤其是“把书放在枕头底下”这种看似荒谬却又有一定“仪式感”的行为，是许多学生都曾尝试或听说过的，因此更容易引发笑声。\n\n3. **结构**：笑话的结构简洁明了，对话形式让情节发展自然流畅。从“绝招”的提出到最后的反转，节奏把握得很好，最后的包袱抖得恰到好处，给人意料之外却又情理之中的感觉。\n\n4. **深度**：虽然是一个简单的笑话，但也隐含了对“临时抱佛脚”这种学习态度的讽刺，提醒人们学习需要脚踏实地，而不是依赖不切实际的方法。\n\n总体而言，这是一个轻松幽默、贴近生活且有一定讽刺意味的笑话，适合在学生群体中传播，能够引发共鸣和笑声。'

In [ ]:
from   langchain_community.vectorstores  import FAISS
from  langchain_core.output_parsers  import  StrOutputParser
from  langchain_core.prompts  import  ChatPromptTemplate
from  langchain_core.runnables  import  RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()

vocterstore = FAISS.from_texts(
    ["张三在华为工作", "张三在上高中"],
    embedding=embedding
)
restiver = vocterstore.as_retriever()
template ="""
    根据上下文回答问题：
    {content}
    
    问题:{question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {'content': restiver,'question': RunnablePassthrough()} | prompt | llm | StrOutputParser()

chain.invoke('张三在哪里工作')

'根据提供的文档内容，张三在华为工作。'

In [7]:
{'content': restiver,'question': RunnablePassthrough()}

{'content': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022F4F87DF30>, search_kwargs={}),
 'question': RunnablePassthrough()}

In [9]:
from   langchain_community.vectorstores  import FAISS
from  langchain_core.output_parsers  import  StrOutputParser
from  langchain_core.prompts  import  ChatPromptTemplate
from  langchain_core.runnables  import  RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from  operator  import itemgetter
embedding = HuggingFaceEmbeddings()

vocterstore = FAISS.from_texts(
    ["张三在华为工作"],embedding=embedding
)
restiver = vocterstore.as_retriever()
template ="""
    根据上下文回答问题：
    {content}
    
    问题:{question}
请用以下语言回答问题：{language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {'content': itemgetter('question') |restiver,
         'question': itemgetter('question'),
         'language': itemgetter('language')
         } | prompt | llm | StrOutputParser()

chain.invoke({'question': '张三在哪里工作','language' : '英语'})

'Zhang San works at Huawei.'

In [11]:
from  langchain_core.runnables  import RunnableParallel

joke_chain = ChatPromptTemplate.from_template("请给我将一个笑话，主题是{topic},字数在50字以内") |llm | StrOutputParser()
poem_chain = ChatPromptTemplate.from_template("请给我写一首诗，主题是{topic},字数在50字以内") |llm | StrOutputParser()

map_chain =RunnableParallel({'joke': joke_chain, 'poem': poem_chain})

map_chain.invoke({'topic': '猫'})

{'joke': '## 《猫的哲学》\n\n"为什么你总是盯着鱼缸看？"主人问。\n\n"我在思考，"猫说，"为什么鱼缸里的鱼总是游得那么快，却永远游不出这个圈。"',
 'poem': '## 《猫》\n\n月光在窗台流淌\n猫的影子\n是夜的裂缝\n\n它用瞳孔丈量\n黑暗的深度\n直到黎明\n在胡须上\n结出露珠'}

In [15]:
from  langchain_core.runnables  import RunnablePassthrough,RunnableParallel

runable = RunnableParallel(
    passed = RunnablePassthrough(),
    modified =lambda x: x['num'] +1
)

runable.invoke({'num':1})

{'passed': {'num': 1}, 'modified': 2}

In [17]:
from  langchain_core.runnables  import RunnablePassthrough,RunnableParallel

runable = RunnableParallel(
    extra = RunnablePassthrough.assign(mul=lambda x: x['num']*3),
    modified =lambda x: x['num'] +1
)

runable.invoke({'num':2})

{'extra': {'num': 2, 'mul': 6}, 'modified': 3}

PromptTemplate 

In [20]:
from langchain.prompts  import PromptTemplate
prompt =PromptTemplate.from_template(
    """
    请给我将一个关于{content}的{adject}笑话,字数在50字以内
    """
)
prompt.format(content='鸡',adject ='活泼的')

'\n    请给我将一个关于鸡的活泼的笑话,字数在50字以内\n    '

ChatPromptTemplate 

In [24]:
from  langchain.prompts  import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个笑话大师,你的名字是joker'),  #  系统信息--->表示它的身份，它是干什么的
        ('human', '请给我将一个关于{content}的{adject}笑话,字数在50字以内'),  # 人类信息--->表示你的需求
        ('ai', '好的,请告诉我{content}和{adject}'),# AI信息--->表示AI的回应
    ]
)
messages = prompt.format_messages(content='鸡',adject ='活泼的')
messages

[SystemMessage(content='你是一个笑话大师,你的名字是joker', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请给我将一个关于鸡的活泼的笑话,字数在50字以内', additional_kwargs={}, response_metadata={}),
 AIMessage(content='好的,请告诉我鸡和活泼的', additional_kwargs={}, response_metadata={})]

In [29]:
from langchain_core.messages import  SystemMessage
from langchain_core.prompts import  HumanMessagePromptTemplate,ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='你是一个笑话大师,你的名字是joker'),  #  系统信息--->表示它的身份，它是干什么的
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)


messages = prompt.format_messages(text='请给我将一个关于鸡的活泼的笑话,字数在50字以内')
messages

[SystemMessage(content='你是一个笑话大师,你的名字是joker', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请给我将一个关于鸡的活泼的笑话,字数在50字以内', additional_kwargs={}, response_metadata={})]